In [2]:
import os
imdb_dir = 'aclImdb'
train_dir = os.path.join(imdb_dir, 'train')
labels = []
texts = []
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [3]:
len(texts)

25000

In [4]:
len(labels)

25000

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
maxlen = 100
training_samples = 200
validation_samples = 10000
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [7]:
word_index = tokenizer.word_index

In [8]:
data = pad_sequences(sequences, maxlen=maxlen)
labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

Shape of data tensor: (25000, 100)
Shape of label tensor: (25000,)


In [9]:
data

array([[1812,  225,   60, ...,    1,  853,  412],
       [  10,  101,    6, ...,  404,  383,    9],
       [  96,    2,  269, ...,  853, 1176,  507],
       ...,
       [ 423,  103,    7, ..., 1595,   12,  122],
       [   8,  809,  238, ...,   97,  867, 3236],
       [1559,  617,    2, ...,    7,  707,  155]], dtype=int32)

In [10]:
len(data[0])

100

In [13]:
labels

array([0, 1, 0, ..., 0, 0, 1])

In [14]:
training_samples

200

In [18]:
glove_dir = ''
embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [20]:
embedding_matrix = np.zeros((max_words, 100))

In [21]:
embedding_matrix.shape

(10000, 100)

In [22]:
embedding_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [32]:
embedding_dim = 100
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          1000000   
_________________________________________________________________
flatten (Flatten)            (None, 10000)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                320032    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 1,320,065
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [35]:
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['acc'])

In [36]:
history = model.fit(x_train, y_train,
epochs=10,
batch_size=32,
validation_data=(x_val, y_val))

Epoch 1/10
7/7 [==============================] - 2s 352ms/step - loss: 1.8002 - acc: 0.5400 - val_loss: 0.7275 - val_acc: 0.5236
Epoch 2/10
7/7 [==============================] - 2s 269ms/step - loss: 0.4114 - acc: 0.8500 - val_loss: 0.7250 - val_acc: 0.5393
Epoch 3/10
7/7 [==============================] - 2s 258ms/step - loss: 0.4104 - acc: 0.8200 - val_loss: 0.9072 - val_acc: 0.5062
Epoch 4/10
7/7 [==============================] - 2s 327ms/step - loss: 0.2469 - acc: 0.9400 - val_loss: 1.4533 - val_acc: 0.5005
Epoch 5/10
7/7 [==============================] - 2s 273ms/step - loss: 0.1989 - acc: 0.9450 - val_loss: 0.7064 - val_acc: 0.5625
Epoch 6/10
7/7 [==============================] - 2s 314ms/step - loss: 0.1221 - acc: 1.0000 - val_loss: 0.7859 - val_acc: 0.5408
Epoch 7/10
7/7 [==============================] - 2s 282ms/step - loss: 0.0801 - acc: 1.0000 - val_loss: 0.8320 - val_acc: 0.5440
Epoch 8/10
7/7 [==============================] - 2s 334ms/step - loss: 0.0795 - acc: 1.00